# Load Image with labels from IDR, analyze using StarDist and compare results

The notebook shows how to load an IDR image with labels.

The image is referenced in the paper "NesSys: a novel method for accurate nuclear segmentation in 3D" published August 2019 in PLOS Biology: https://doi.org/10.1371/journal.pbio.3000388 and can be viewed online in the [Image Data Resource](https://idr.openmicroscopy.org/webclient/?show=image-6001247).


In this notebook, the image is loaded together with the labels and analyzed using [StarDist](https://stardist.net/docs/index.html). The StarDist analysis produces a segmentation, which is then viewed side-by-side with the original segmentations produced by the authors of the paper obtained via the loaded labels. 

### Install dependencies if required

The cell below will install dependencies if you choose to run the notebook in [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb#recent=true). 

In [1]:
%pip install omero-py stardist geojson

Note: you may need to restart the kernel to use updated packages.


### Import 

In [1]:
from omero.gateway import BlitzGateway

import matplotlib.pyplot as plt
%matplotlib inline
import numpy

### Create a connection to IDR

In [2]:
HOST = 'ws://idr.openmicroscopy.org/omero-ws'
conn = BlitzGateway('public', 'public',
                    host=HOST, secure=True)
print(conn.connect())
conn.c.enableKeepAlive(60)

True


### IDR image to analyze

In [3]:
image_id = 6001247

In [4]:
image = conn.getObject("Image", image_id)

### Helper method to load the 5D image

The image is loaded a TCZYX numpy array to match the Zarr file.

In [5]:
def load_numpy_array(image):
    pixels = image.getPrimaryPixels()
    size_z = image.getSizeZ()
    size_c = image.getSizeC()
    size_t = image.getSizeT()
    size_y = image.getSizeY()
    size_x = image.getSizeX()
    z, t, c = 0, 0, 0  # first plane of the image

    zct_list = []
    for t in range(size_t):
        for c in range(size_c):  # all channels
            for z in range(size_z):  # get the Z-stack
                zct_list.append((z, c, t))

    values = []
    # Load all the planes as YX numpy array
    planes = pixels.getPlanes(zct_list)
    s = "t:%s c:%s z:%s y:%s x:%s" % (size_t, size_c, size_z, size_y, size_x)
    print(s)
    print("Downloading image %s" % image.getName())
    all_planes = numpy.stack(list(planes))
    shape = (size_t, size_c, size_z, size_y, size_x)
    return numpy.reshape(all_planes, newshape=shape)

### Load the data 

In [6]:
data = load_numpy_array(image)

t:1 c:2 z:257 y:210 x:253


### Load the labels

Labels have been saved as mask.

In [8]:
from omero_zarr import masks

In [9]:
roi_service = conn.getRoiService()
result = roi_service.findByImage(image_id, None)

dims = (image.getSizeT(), image.getSizeC(), image.getSizeZ(), image.getSizeY(), image.getSizeX())
shapes = []
for roi in result.rois:
    shapes.append(roi.copyShapes())

saver = masks.MaskSaver(None, image, numpy.int64)
labels, fillColors, properties = saver.masks_to_labels(shapes, mask_shape=dims)

In [10]:
print(labels.shape)

(1, 2, 257, 210, 253)


## Load stardist trained model 

In [11]:
from stardist.models import StarDist2D
model_versatile = StarDist2D.from_pretrained('2D_demo')

Found model '2D_demo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.486166, nms_thresh=0.5.


2022-03-31 15:36:56.446321: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Prediction based on a default StarDist model
Normalize the input image

``model_versatile.predict_instances`` will

 * predict object probabilities and star-convex polygon distances (see model.predict if you want those)
 * perform non-maximum suppression (with overlap threshold nms_thresh) for polygons above object probability threshold prob_thresh.
 * render all remaining polygon instances in a label image
 * return the label instances image and also the details (coordinates, etc.) of all remaining polygons

In [80]:
from csbdeep.utils import normalize
from geojson import Feature, FeatureCollection, Polygon
axis_norm = (0,1)
img = normalize(data[0, 0, :, :, :], 1,99.8, axis=axis_norm)
results = []
shapes = []
c = 0
for i in range(len(img)):
    new_labels, details = model_versatile.predict_instances(img[i])
    # Convert into Polygon and add to Geometry Collection
    for obj_id, region in enumerate(details['coord']):
        coordinates = []
        x = region[1]
        y = region[0]
        for j in range(len(x)):
            coordinates.append((float(x[j]), float(y[j])))
        # append the first coordinate to close the polygon
        coordinates.append(coordinates[0])
        shape = Polygon(coordinates)
        properties = {
            "stroke-width": 1,
            "z": i,
            "c": c,
        }
        shapes.append(Feature(geometry=shape, properties=properties))    
    results.append(new_labels)

label_slices = numpy.array(results)
gc = FeatureCollection(shapes)

## Save the Stardist labels
Save the labels locally in a machine and human readable format: geojson.
Stardist offers method to save the labels into ImageJ rois using ``export_imagej_rois``. This is outside the scope of this notebook.


In [81]:
# Save the shapes as geojson
import geojson
geojson_file = "stardist_shapes_%s.geojson" % image_id
geojson_dump = geojson.dumps(gc, sort_keys=True)
with open(geojson_file, 'w') as out:
    out.write(geojson_dump)

## Compare labels
Display the original labels and the labels based on StarDist prediction side-by-side

In [14]:
from ipywidgets import *

def update(z=0):
    c = 0
    fig = plt.figure(figsize=(10, 10))
    plt.subplot(121)
    plt.imshow(data[0, c, z, :, :], cmap='gray')
    try:
        plt.imshow(labels[0, c, z, :, :], cmap='jet', alpha=0.5)
    except Exception:
        print(z)
    plt.subplot(122)
    plt.imshow(data[0, c, z, :, :], cmap='gray')
    plt.imshow(label_slices[z, :, :], cmap='jet', alpha=0.5)
    plt.tight_layout()
    fig.canvas.flush_events()

interact(update, z= widgets.IntSlider(value=1, min=0, max=data.shape[2]-1, step=1, description="Select Z", continuous_update=False))

interactive(children=(IntSlider(value=1, continuous_update=False, description='Select Z', max=256), Output()),…

<function __main__.update(z=0)>

### Close the connection  

In [13]:
conn.close()

## Exercises

**Exercise 1:**
 - Using the json library, read the geojson file into a variable in this notebook
 - Display the shapes on a selected z-section e.g. z = 5.

**Exercise 2:**
 - Convert the Stardist labels into OMERO polygons.
 - Save the converted labels to an OMERO.server if possible.
 
 
See [Solutions](Solution_Exercises.ipynb)

### License (BSD 2-Clause)
Copyright (C) 2022 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer.
Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution.
THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. 